In [ ]:
# This notebook demonstrates how to calculate the transofmration matrix by 3D Zernike moments using ZMPY3D with PyTorch.
#
# This notebook primarily consists of the following steps:
#     1. Install ZMPY3D_PT and py3Dmol (visualisation).
#     2. Download example PDB data with coordinates.
#     3. Define necessary parameters.
#     4. Load precalculated cache.
#     5. Convert coordinate data into a voxel.
#     6. Create a callable function for generating Zernike moments and normalization.
#     7. Obtain the transform matrix.
#     8. A command line interface (CLI) example

In [ ]:
# Install ZMPY3D versions for PyTorch.
# Install py3Dmol for visualisation.

! pip install ZMPY3D-PT
! pip install py3Dmol

print(f"It is recommended to restart the Python kernel for the IPython notebook.")

In [ ]:
# # Download example data from GitHub using curl
! curl -OJL https://github.com/tawssie/ZMPY3D/raw/main/6NT5.pdb
! curl -OJL https://github.com/tawssie/ZMPY3D/raw/main/6NT6.pdb


In [ ]:
import ZMPY3D_PT as z
import torch
import numpy as np

if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("CUDA is available, PyTorch uses GPU.")
else:
    print("CUDA is unavailable. PyTorch uses CPU.")

Param=z.get_global_parameter()
ResidueBox=z.get_residue_gaussian_density_cache(Param)
GridWidth = 1.0

# Convert structure data into coordinates
[XYZ_A,AA_NameList_A]=z.get_pdb_xyz_ca('./6NT5.pdb')
# Convert coordinates into voxels using precalculated Gaussian densities
[Voxel3D_A,CornerA]=z.fill_voxel_by_weight_density(XYZ_A,AA_NameList_A,Param['residue_weight_map'],GridWidth,ResidueBox[GridWidth])

Voxel3D_A = torch.tensor(Voxel3D_A, dtype=torch.float64)
CornerA =torch.tensor(CornerA,dtype=torch.float64)

print(f"Structure A (6NT5) has been converted into a gridded voxel with the shape {Voxel3D_A.shape}")

[XYZ_B,AA_NameList_B]=z.get_pdb_xyz_ca('./6NT6.pdb')
[Voxel3D_B,CornerB]=z.fill_voxel_by_weight_density(XYZ_B,AA_NameList_B,Param['residue_weight_map'],GridWidth,ResidueBox[GridWidth])

Voxel3D_B=torch.tensor(Voxel3D_B,dtype=torch.float64)
CornerB =torch.tensor(CornerB,dtype=torch.float64)

print(f"Structure B (6NT6) has been converted into a gridded voxel with the shape {Voxel3D_B.shape}")



In [ ]:
import pickle

MaxOrder=6

# Find the cache_data directory based on the site package location of ZMPY3D_PT.
BinomialCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/BinomialCache.pkl'
with open(BinomialCacheFilePath, 'rb') as file:
    BinomialCachePKL = pickle.load(file)

LogCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/LogG_CLMCache_MaxOrder{:02d}.pkl'.format(MaxOrder)

with open(LogCacheFilePath, 'rb') as file:
    CachePKL = pickle.load(file)

BinomialCache=torch.tensor(BinomialCachePKL['BinomialCache'])

GCache_pqr_linear = torch.tensor(CachePKL['GCache_pqr_linear'])
GCache_complex = torch.tensor(CachePKL['GCache_complex'])
GCache_complex_index = torch.tensor(CachePKL['GCache_complex_index'])
CLMCache3D = torch.tensor(CachePKL['CLMCache3D'], dtype=torch.complex128)
CLMCache = torch.tensor(CachePKL['CLMCache'], dtype=torch.float64)

RotationIndex=CachePKL['RotationIndex']

# RotationIndex is a structure, must be [0,0] to accurately obtain the s_id ... etc, within RotationIndex.
s_id = torch.tensor(np.squeeze(RotationIndex['s_id'][0,0]) - 1, dtype=torch.int64)
n    = torch.tensor(np.squeeze(RotationIndex['n'][0,0]), dtype=torch.int64)
l    = torch.tensor(np.squeeze(RotationIndex['l'][0,0]), dtype=torch.int64)
m    = torch.tensor(np.squeeze(RotationIndex['m'][0,0]), dtype=torch.int64)
mu   = torch.tensor(np.squeeze(RotationIndex['mu'][0,0]), dtype=torch.int64)
k    = torch.tensor(np.squeeze(RotationIndex['k'][0,0]), dtype=torch.int64)
IsNLM_Value = torch.tensor(np.squeeze(RotationIndex['IsNLM_Value'][0,0]) - 1, dtype=torch.int64)


In [ ]:

def OneTimeConversion(Voxel3D,Corner,GridWidth,BinomialCache, CLMCache, CLMCache3D, GCache_complex, GCache_complex_index, GCache_pqr_linear, MaxOrder, Param, ResidueBox, RotationIndex):

    Dimension_BBox_scaled=Voxel3D.shape;

    X_sample = torch.arange(0, Dimension_BBox_scaled[0] + 1, dtype=torch.float64)
    Y_sample = torch.arange(0, Dimension_BBox_scaled[1] + 1, dtype=torch.float64)
    Z_sample = torch.arange(0, Dimension_BBox_scaled[2] + 1, dtype=torch.float64)

    # Calculate the volume mass and the center of mass
    [VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,X_sample,Y_sample,Z_sample)

    [AverageVoxelDist2Center,MaxVoxelDist2Center]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,1.80) # Param['default_radius_multiplier'] == 1.80

    Center_scaled=Center*GridWidth+Corner

    ##################################################################################
    # You may add any preprocessing on the voxel before applying the Zernike moment. #
    ##################################################################################

    # Calculate the weights for sphere sampling
    Sphere_X_sample, Sphere_Y_sample, Sphere_Z_sample=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)

    # Apply weights to the geometric moments
    _,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D
                                      ,MaxOrder
                                      ,Sphere_X_sample
                                      ,Sphere_Y_sample
                                      ,Sphere_Z_sample)

    # Convert to unscaled 3D Zernike moments
    _,ZMoment_raw=z.calculate_bbox_moment_2_zm(MaxOrder
                                       , GCache_complex
                                       , GCache_pqr_linear
                                       , GCache_complex_index
                                       , CLMCache3D
                                       , SphereBBoxMoment)


    # Calculate alternative 3D Zernike moments for specific normalisation orders 2, 3, 4, 5 and 6
    ABList_2=z.calculate_ab_rotation_all(ZMoment_raw, 2)
    ABList_3=z.calculate_ab_rotation_all(ZMoment_raw, 3)
    ABList_4=z.calculate_ab_rotation_all(ZMoment_raw, 4)
    ABList_5=z.calculate_ab_rotation_all(ZMoment_raw, 5)
    ABList_6=z.calculate_ab_rotation_all(ZMoment_raw, 6)


    ABList_all = torch.cat([torch.reshape(ABList_2, (-1, 2)),torch.reshape(ABList_3, (-1, 2)),torch.reshape(ABList_4, (-1, 2)),torch.reshape(ABList_5, (-1, 2)),torch.reshape(ABList_6, (-1, 2))], dim=0)
    ZMList_all = z.calculate_zm_by_ab_rotation(ZMoment_raw, BinomialCache, ABList_all, MaxOrder, CLMCache,s_id,n,l,m,mu,k,IsNLM_Value)
    ZMList_all = ZMList_all[~torch.isnan(ZMList_all.real)]
    ZMList_all = torch.reshape(ZMList_all, (-1, 96))

    return Center_scaled, ABList_all,ZMList_all

print(f"Merge all steps into a single callable function, OneTimeConversion, generating alternative 3D Zernike moments for rotational variations")


In [ ]:
%%time
# Compute all possible rotations
Center_scaled_A,ABList_A,ZMList_A =OneTimeConversion(Voxel3D_A,CornerA,1.00,BinomialCache, CLMCache, CLMCache3D, GCache_complex, GCache_complex_index, GCache_pqr_linear, MaxOrder, Param, ResidueBox, RotationIndex)
Center_scaled_B,ABList_B,ZMList_B =OneTimeConversion(Voxel3D_B,CornerB,1.00,BinomialCache, CLMCache, CLMCache3D, GCache_complex, GCache_complex_index, GCache_pqr_linear, MaxOrder, Param, ResidueBox, RotationIndex)

# Compare all Zernike moments and select the maximum value using a dot product calculation

M = torch.abs(torch.matmul(torch.conj(ZMList_A).t(), ZMList_B)) # square matrix A^T*B

MaxValueIndex = torch.nonzero(M == torch.max(M)) # MaxValueIndex is a tuple that contains an nd array.

i,j=MaxValueIndex[0,0], MaxValueIndex[0,1]

# Compute the transformation matrix for protein structure A
RotM_A=z.get_transform_matrix_from_ab_list(ABList_A[i,0],ABList_A[i,1],Center_scaled_A)
RotM_B=z.get_transform_matrix_from_ab_list(ABList_B[j,0],ABList_B[j,1],Center_scaled_B)

TargetRotM = torch.linalg.solve(RotM_B, RotM_A)

print(f"Here is the transformation matrix and computation time provided by ZMPY3D_PT for protein structure A.")
print(TargetRotM)


In [ ]:

# "SetPDB_XYZ_RotM is not a standard function; it merely applies a matrix to atom coordinates.
# It is recommended to use BioPython for this purpose."
z.set_pdb_xyz_rot('./6NT5.pdb',TargetRotM.cpu().numpy(),'6NT5_trans.pdb')

print(f"Apply the transformation matrix to all coordinates of protein structure A (6NT5.pdb) and save the modified structure as 6NT5_trans.pdb.")

In [ ]:
import py3Dmol

view = py3Dmol.view(width=400, height=400)

def add_model_to_viewer(file_path, model_style):
    with open(file_path, 'r') as file:
        pdb_data = file.read()
    view.addModel(pdb_data, 'pdb')
    view.setStyle({'model': -1}, model_style)

add_model_to_viewer('6NT5.pdb', {'cartoon': {'color': 'spectrum', 'opacity': 0.6}})
add_model_to_viewer('6NT6.pdb', {'cartoon': {'color': 'gray', 'opacity': 0.8}})

view.zoomTo()

print("")
print("Colored 6NT5 (structure A) and gray 6NT6 (structure B) before the superposition.")
view.show()


In [ ]:
import py3Dmol

view = py3Dmol.view(width=400, height=400)

def add_model_to_viewer(file_path, model_style):
    with open(file_path, 'r') as file:
        pdb_data = file.read()
    view.addModel(pdb_data, 'pdb')
    view.setStyle({'model': -1}, model_style)

add_model_to_viewer('6NT5_trans.pdb', {'cartoon': {'color': 'spectrum', 'opacity': 0.6}})
add_model_to_viewer('6NT6.pdb', {'cartoon': {'color': 'gray', 'opacity': 0.8}})

view.zoomTo()

print("")
print("Colored 6NT5_trans (transformed structure A) and gray 6NT6 (structure B) after the superposition.")
view.show()

In [ ]:
# Alternatively, use a system call to compute results via CLI
# ./ZMPY3D_PT_CLI_SuperA2B PDB_A PDB_B
! ZMPY3D_PT_CLI_SuperA2B "./6NT5.pdb" "./6NT6.pdb"
